In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import yaml
import h5py
import os
import datetime
import gc
from tqdm import tqdm

from torch.utils.data import Dataset

In [2]:
path_uplink = "../DeepMIMO/DeepMIMO Dataset/DeepMIMO_dataset_I_2p4_1_ofdm_128_yant.mat"
path_downlink = "../DeepMIMO/DeepMIMO Dataset/DeepMIMO_dataset_I_2p5_1_ofdm_128_yant.mat"

path_to_uplink_npy = "../DeepMIMO/DeepMIMO Dataset/dataset_I_2p4_1_ofdm_128_yant"
path_to_downlink_npy = "../DeepMIMO/DeepMIMO Dataset/dataset_I_2p5_1_ofdm_128_yant"

f_uplink = h5py.File(path_uplink, 'r')
f_downlink = h5py.File(path_downlink, 'r')

In [3]:
class DeepMIMODatasetForUPA(Dataset):
    def __init__(self, f1, f2, user_indices=None, is_location=False):
        """
        params: f1: A h5py file object for uplink.
                f2: A h5py file object for downlink.
                indices: indices of users that will be inculded
                         None means all users.
        Returns: Dictionary containing 'uplink' and 'downlink' as keys
                uplink: contains list where each element represents channel gain for specific basestation
                downlink: contains list where each element represents channel gain for specific basestation
        """
        self.f_uplink = f1
        self.f_downlink = f2
        self.user_indices = user_indices
        self.bs_data_uplink = self.f_uplink['DeepMIMO_dataset']      # 'DeepMIMO_dataset' is the default key.
        self.bs_data_downlink = self.f_downlink['DeepMIMO_dataset']
        self.num_active_bs_uplink, _ = self.bs_data_uplink.shape
        self.num_active_bs_downlink, _ = self.bs_data_downlink.shape
        self.is_location = is_location
    
    def __len__(self):
        if self.user_indices is None:
            return self.f_uplink[self.bs_data_uplink[0][0]]['user'].shape[0]    # returns number of users.
        else:
            return len(self.user_indices)
    
    def __getitem__(self, index):
        # for uplink
        specific_user_data_uplink = []
        for active_bs_index in range(self.num_active_bs_uplink):
            data_uplink_per_bs = []
            user_data = self.f_uplink[self.bs_data_uplink[active_bs_index][0]]['user']
            channel_data = self.f_uplink[user_data[index][0]]['channel'] # ** code debug **
            num_carriers, num_ant = channel_data.shape
            for ant_index in range(num_ant):
                channel_gain = channel_data[0][ant_index][0] + channel_data[0][ant_index][1]*1j
                data_uplink_per_bs.append(channel_gain)
            specific_user_data_uplink.append(data_uplink_per_bs)
        specific_user_data_uplink = np.array(specific_user_data_uplink)
        
        # for downlink
        specific_user_data_downlink = []
        for active_bs_index in range(self.num_active_bs_downlink):
            data_downlink_per_bs = []
            user_data = self.f_downlink[self.bs_data_downlink[active_bs_index][0]]['user']
            channel_data = self.f_downlink[user_data[index][0]]['channel'] # ** code debug **
            num_carriers, num_ant = channel_data.shape
            for ant_index in range(num_ant):
                channel_gain = channel_data[0][ant_index][0] + channel_data[0][ant_index][1]*1j
                data_downlink_per_bs.append(channel_gain)
            specific_user_data_downlink.append(data_downlink_per_bs)
        specific_user_data_downlink = np.array(specific_user_data_downlink)
        
        return {"uplink": specific_user_data_uplink, "downlink": specific_user_data_downlink}

In [4]:
dataset = DeepMIMODatasetForUPA(f_uplink, f_downlink)

In [5]:
num_users = len(dataset)
uplink_dim = (num_users,128*2)
downlink_dim = (num_users,128*2)

uplink_channel_gain = np.zeros(uplink_dim)
downlink_channel_gain = np.zeros(downlink_dim)

for i in tqdm(range(num_users), total=num_users):
    x = dataset[i]
    uplink_channel_gain[i,:] = np.hstack((x['uplink'].real, x['uplink'].imag))
    downlink_channel_gain[i,:] = np.hstack((x['downlink'].real, x['downlink'].imag))

100%|██████████| 151402/151402 [4:51:32<00:00,  9.46it/s]  


In [6]:
%%time
np.save(path_to_uplink_npy, uplink_channel_gain)
np.save(path_to_downlink_npy, downlink_channel_gain)

CPU times: user 63.7 ms, sys: 337 ms, total: 400 ms
Wall time: 588 ms
